In [2]:
import graphlab as gl
import numpy as np
import matplotlib.pyplot as plt


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1489275156.log


This non-commercial license of GraphLab Create for academic use is assigned to victor.li.zhu@rutgers.edu and will expire on October 06, 2017.


In [3]:
def sig(t):
    sig = 1. / (1+np.exp(-t))
    return sig

In [3]:
# comb = gl.SFrame.read_csv('/Users/lizhu/Dropbox/projects/calcium/VG_result/combined_feature_vectors_windowLen300_winStep_050_v2_threshold_10.csv', delimiter=',',header=False,verbose = False)
comb = gl.SFrame.read_csv('/Users/lizhu/Dropbox/projects/calcium/format4ML/format4ML_GC6f_emx_06_windowLen200_winStep_050_v2_threshold_10.csv', delimiter=',',header=False,verbose = False)
colName_dg = 'degree'
colName_dg = gl.SArray([colName_dg + repr(i+1) for i in range(30)])

colName_cc = 'clusteringCoef'
colName_cc = gl.SArray([colName_cc + repr(i+1) for i in range(30)])

colName_pl = 'pathlength'
colName_pl = gl.SArray([colName_pl + repr(i+1) for i in range(30)])

colName = colName_dg.append(colName_cc.append(colName_pl.append(gl.SArray(['Whisker_lab','Whisker_cont','Pupil_lab','Pupil_cont' ]))))
colName = (list(colName))


dictionary = dict(zip(comb.column_names(), colName))
comb = comb.rename(dictionary)
comb = gl.toolkits.cross_validation.shuffle(comb, random_seed=1)

comb['Whisker_lab'].show(view = 'Categorical')

Canvas is accessible via web browser at the URL: http://localhost:56881/index.html
Opening Canvas in default web browser.


In [4]:
# NOTE：run the functions below before running this section!!!
feature = comb.column_names()[0:90]
# feature 1~30: degree, 31~60: clustering coefficient, 60~90: pathlength
label = 'Whisker_lab'
l2_penalty = 0.05
# acc,SE,SP = lz_logistic(comb, feature, label, l2_penalty)

In [5]:
folds = gl.cross_validation.KFold(comb, 5)
SE=[None]*5
SP=[None]*5
acc=[None]*5
# print specificity

data_train = comb[0:100]
data_test  = comb[100:110]

m = gl.logistic_classifier.create(data_train,
                              target=label,
                              features=feature,
                              l2_penalty = l2_penalty,
                              validation_set=None,
                              verbose = False)
results          = m.evaluate(data_test)

In [11]:
roc = results['roc_curve']
# roc = roc.to_numpy()

# false_positive_rate
x = roc['fpr']
# true_positive_rate 
y = roc['tpr']

print (x.shape)
# plt.plot(x,y)
# plt.show() 

# This is the AUC
auc = np.trapz(y,x)
range(6)

(100001,)


[0, 1, 2, 3, 4, 5]

In [5]:
def get_numpy_data(data_sframe, features, label):
    data_sframe['intercept'] = 1
    features = ['intercept'] + features
    features_sframe = data_sframe[features]
    feature_matrix = features_sframe.to_numpy()
    label_sarray = data_sframe[label]
    label_array = label_sarray.to_numpy()
    return(feature_matrix, label_array)

In [5]:
############### this results have been used for CLEO paper

# comb = gl.toolkits.cross_validation.shuffle(comb, random_seed=2)

def lz_logistic(data, feature, label, l2_penalty):
    
    folds = gl.cross_validation.KFold(comb, 5)
    SE=[None]*5
    SP=[None]*5
    acc=[None]*5
    # print specificity
    idx = 0
    for train, valid in folds:
        m = gl.svm_classifier.create(train,
                                          target=label,
                                          features=feature,
                                          l2_penalty = l2_penalty,
                                          validation_set=None,
                                          verbose = False)
        results          = m.evaluate(data_test, 'confusion_matrix')
        confusion_matrix = results['confusion_matrix']
        confusion_matrix = m.evaluate(valid, 'confusion_matrix')
        confusion_matrix = confusion_matrix.values()[0]
        TP = confusion_matrix[(confusion_matrix['target_label']==1) & (confusion_matrix['predicted_label']==1)]
        if np.size(TP) == 0:
            TP = 0
        else:
            TP = TP['count'][0]
        TN = confusion_matrix[(confusion_matrix['target_label']==0) & (confusion_matrix['predicted_label']==0)]
        if np.size(TN) == 0:
            TN = 0
        else:
            TN = TN['count'][0]
        FP = confusion_matrix[(confusion_matrix['target_label']==0) & (confusion_matrix['predicted_label']==1)]
        if np.size(FP) == 0:
            FP = 0
        else:
            FP = FP['count'][0]
        FN = confusion_matrix[(confusion_matrix['target_label']==1) & (confusion_matrix['predicted_label']==0)]
        if np.size(FN) == 0:
            FN = 0
        else:
            FN = FN['count'][0]
        SP[idx] = float(TN) / (TN + FP)
        SE[idx] = float(TP) / (TP + FN)
        acc[idx] = float(TP+TN) / (TP+TN+FP+FN)
        idx = idx + 1
    
    print 'acc_mean = ', np.mean(acc)
    print 'acc_std = ', np.std(acc)
    print 'SE_mean = ', np.mean(SE)
    print 'SE_std = ', np.std(SE)
    print 'SP_mean = ', np.mean(SP)
    print 'SP_std = ', np.std(SP)
    
    return results, acc, SE, SP

In [10]:
############### this is the normalized version of the above code

# normalize the features
# feature_matrix is a SFrame, first 90 columns are features, last column is labels.
def normalize_features_mice(comb):
    # extract feature columns
    comb_np = comb.to_numpy()
    comb_feature = comb_np[:,0:90]
    
    # compute normed features and norms 
    norms = np.linalg.norm(comb_feature, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
    comb_feature_normed = comb_feature / norms
    
    # combine normalized feature vectors and the labels
    comb_normed = np.concatenate((comb_feature_normed,comb_np[:,90:91]),axis = 1)
    # convert numpy array to SFrame
    comb_normed = gl.SFrame(comb_normed)
    comb_normed = comb_normed.unpack('X1', '')
    dictionary_norm = dict(zip(comb_normed.column_names(), colName))
    comb_normed = comb_normed.rename(dictionary_norm) # this is the normed version

    return comb_normed, norms # note: norms is array, comb_normed is SFrame

# comb = gl.toolkits.cross_validation.shuffle(comb, random_seed=2)
def lz_logistic(data, feature):
    
    # normalize 
    comb, norms = normalize_features_mice(data)
    comb['Whisker'] = comb['Whisker'].astype(int)

    folds = gl.cross_validation.KFold(comb, 5)
    SE=[None]*5
    SP=[None]*5
    acc=[None]*5
    # print specificity
    idx = 0
    for train, valid in folds:
        m = gl.logistic_classifier.create(train,
                                          target='Whisker',
                                          features=feature,
                                          l2_penalty = 0.05,
                                          validation_set=None,
                                          verbose = False)
        
        # normalize the validation dataset
        valid_np = valid.to_numpy()
        valid_feature = valid_np[:,0:90]

        # use normed from features
        valid_feature_normed = valid_feature / norms

        # combine normalized feature vectors and the labels
        valid_normed = np.concatenate((valid_feature_normed,valid_np[:,90:91]),axis = 1)

        # convert numpy array to SFrame
        valid_normed = gl.SFrame(valid_normed)
        valid_normed = valid_normed.unpack('X1', '')
        dictionary_norm = dict(zip(valid_normed.column_names(), colName))
        valid_normed = valid_normed.rename(dictionary_norm) # this is the normed version
        valid_normed['Whisker'] = valid_normed['Whisker'].astype(int)

        confusion_matrix = m.evaluate(valid, 'confusion_matrix')
        confusion_matrix = confusion_matrix.values()[0]
        TP = confusion_matrix[(confusion_matrix['target_label']==1) & (confusion_matrix['predicted_label']==1)]
        if np.size(TP) == 0:
            TP = 0
        else:
            TP = TP['count'][0]
        TN = confusion_matrix[(confusion_matrix['target_label']==0) & (confusion_matrix['predicted_label']==0)]
        if np.size(TN) == 0:
            TN = 0
        else:
            TN = TN['count'][0]
        FP = confusion_matrix[(confusion_matrix['target_label']==0) & (confusion_matrix['predicted_label']==1)]
        if np.size(FP) == 0:
            FP = 0
        else:
            FP = FP['count'][0]
        FN = confusion_matrix[(confusion_matrix['target_label']==1) & (confusion_matrix['predicted_label']==0)]
        if np.size(FN) == 0:
            FN = 0
        else:
            FN = FN['count'][0]
        SE[idx] = float(TN) / (TN + FP)
        SP[idx] = float(TP) / (TP + FN)
        acc[idx] = float(TP+TN) / (TP+TN+FP+FN)
        idx = idx + 1
    
    print 'acc_mean = ', np.mean(acc)
    print 'acc_std = ', np.std(acc)
    print 'SE_mean = ', np.mean(SE)
    print 'SE_std = ', np.std(SE)
    print 'SP_mean = ', np.mean(SP)
    print 'SP_std = ', np.std(SP)
    
    return acc, SE, SP

# normalize

feature = comb.column_names()[60:90] 

acc,SE,SP = lz_logistic(comb, feature)



acc_mean =  0.889285714286
acc_std =  0.0165600330277
SE_mean =  0.965258005373
SE_std =  0.0144718542275
SP_mean =  0.364412254412
SP_std =  0.0383172930455


acc_mean =  0.941964285714
acc_std =  0.00747017880834
SE_mean =  0.662522477522
SE_std =  0.0484913858058
SP_mean =  0.981643612402
SP_std =  0.00598084605908


In [894]:
# exploring the data
num_wm = len(comb[comb['Whisker']==1])
num_wn = len(comb[comb['Whisker']==0])
# major class classifier(naive classifier)
major_classifier = float(num_wn)/(num_wm + num_wn)
major_classifier
num_wn

393

In [414]:
train_data,test_data = comb.random_split(.8, seed=0)
# train_valid_shuffled = graphlab.toolkits.cross_validation.shuffle(train_valid, random_seed=1)
train_data = gl.toolkits.cross_validation.shuffle(train_data, random_seed=1)

In [678]:
folds = gl.cross_validation.KFold(comb, 5)
folds
params = dict([('target', 'Whisker'), ('features', comb.column_names()[0:90]), ('validation_set', None)])

all_feature_model = gl.cross_validation.cross_val_score(folds,
                                              gl.logistic_classifier.create,
                                              params)

[INFO] graphlab.deploy.job: Validating job.
[INFO] graphlab.deploy.map_job: Validation complete. Job: 'Cross-Validation-Dec-10-2016-13-00-57-052893-23794864' ready for execution
[INFO] graphlab.deploy.map_job: Job: 'Cross-Validation-Dec-10-2016-13-00-57-052893-23794864' scheduled.


In [679]:
# print all_feature_model.get_results()
all_feature_model.get_results()

[INFO] graphlab.deploy._job: Waiting for job to finish, this may take quite a while.
[INFO] graphlab.deploy._job: You may CTRL-C to stop this command and it will not cancel your job.


OrderedDict([('models', [Class                          : LogisticClassifier
               
               Schema
               ------
               Number of coefficients         : 91
               Number of examples             : 448
               Number of classes              : 2
               Number of feature columns      : 90
               Number of unpacked features    : 90
               
               Hyperparameters
               ---------------
               L1 penalty                     : 0.0
               L2 penalty                     : 0.01
               
               Training Summary
               ----------------
               Solver                         : newton
               Solver iterations              : 6
               Solver status                  : SUCCESS: Optimal solution found.
               Training time (sec)            : 0.3326
               
               Settings
               --------
               Log-likelihood           

In [395]:
mylist = sum([range(0,30)] + [range(60,90)],[])
mylist[0:3,5]

TypeError: list indices must be integers, not tuple

In [215]:
type(colName)

list

In [132]:
whisker_model.evaluate(test_data)

{'accuracy': 0.928, 'auc': 0.9809523809523802, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        0        |   87  |
 |      0       |        1        |   3   |
 |      1       |        0        |   6   |
 |      1       |        1        |   29  |
 +--------------+-----------------+-------+
 [4 rows x 3 columns], 'f1_score': 0.8656716417910447, 'log_loss': 0.17156060617233557, 'precision': 0.90625, 'recall': 0.8285714285714286, 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+----------------+-----+----+----+
 | threshold |      fpr       | tpr | p  | n  |
 +-----------+----------------+-----+----+----+
 |    0.0    |      1.0       | 1.0 | 35 | 90 |
 |   1e-05   | 0.988888888889 | 1.0 | 35 | 90 |
 |   2e-05   |

In [148]:
## Weight 
weight = whisker_model['coefficients'].sort('value', ascending=False)
weight['name']

dtype: str
Rows: 91
['degree10', 'degree22', 'degree16', 'degree9', 'degree20', 'degree30', 'degree24', 'degree23', 'degree28', 'degree12', 'degree11', 'degree2', '(intercept)', 'degree14', 'degree29', 'clusteringCoef23', 'clusteringCoef22', 'pathlength6', 'clusteringCoef9', 'pathlength14', 'clusteringCoef27', 'clusteringCoef18', 'clusteringCoef2', 'pathlength21', 'clusteringCoef14', 'clusteringCoef8', 'clusteringCoef13', 'clusteringCoef19', 'pathlength23', 'pathlength11', 'clusteringCoef5', 'pathlength24', 'pathlength26', 'clusteringCoef3', 'pathlength20', 'pathlength1', 'pathlength22', 'clusteringCoef17', 'clusteringCoef28', 'clusteringCoef26', 'clusteringCoef29', 'pathlength3', 'pathlength30', 'clusteringCoef24', 'pathlength25', 'clusteringCoef11', 'pathlength2', 'pathlength9', 'pathlength27', 'pathlength17', 'pathlength18', 'pathlength16', 'pathlength5', 'pathlength19', 'pathlength7', 'clusteringCoef1', 'pathlength8', 'clusteringCoef7', 'pathlength10', 'pathlength28', 'pathlength13

In [149]:
## 